# QCTSP-Hawkes: Classical vs QCTSP Comparison

目的: 古典 Hawkes と QCTSP(Hawkes) の比較を行い、分布指標を可視化します。

環境:
- Python: `/Users/kencharoff/workspace/envs/qc/.venv/bin/python`
- 主要依存: numpy, pandas, matplotlib, qiskit

出力:
- `outputs/compare_report.json`
- `outputs/figures/qctsp_vs_classical_nt_hist.png`
- `outputs/figures/qctsp_vs_classical_integrals.png`

追加出力(実データ):
- `outputs/real_data/real_data_report.json`
- `outputs/real_data/real_data_meta.json`
- `outputs/figures/real_data_nt_hist.png`
- `outputs/figures/real_data_integrals.png`



## 初学者向けの読み方

- **Hawkes過程**は「イベントが起きると、しばらく次のイベントが起きやすくなる」という自己励起型モデルです。
  例: 取引が成立すると次の取引が続きやすい、といった現象を説明します。
- **QCTSP**は連続時間の確率過程を量子回路で模擬する枠組みです。
- このノートの目的は、**古典シミュレーション**と**QCTSP**の出力統計が
  どの程度一致するかを確認することです。


In [ ]:
# Setup: imports and project root resolution
from __future__ import annotations

import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image, display


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'src' / 'qctsp_hawkes').exists():
            return candidate
    raise RuntimeError('project root not found (src/qctsp_hawkes missing)')


project_root = find_project_root(Path.cwd())
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

project_root


In [ ]:
# Parameters
T = 10.0
mu = 0.8
alpha = 0.6
beta = 1.5

S = 8
R = 16
tau_max = 5.0
eta_max = 8.0

n_steps = 3
shots = 200
classical_paths = 200
seed = 7
x0 = 0

# qctsp_backend: 'classical' / 'auto' / 'aer'
qctsp_backend = 'classical'
auto_n_steps = True

# discretization modes
tau_repr_mode = 'midpoint'   # or 'condexp'
tau_edges_mode = 'linear'    # or 'exp-quantile'
lambda_ref_mode = 'mu'       # or 'lambda_x0', 'lambda_mean'

output_dir = project_root / 'outputs'
figure_dir = output_dir / 'figures'
figure_dir.mkdir(parents=True, exist_ok=True)
output_dir


In [ ]:
# Real-data parameters (OHLCV CSV)
real_data_subdir = 'price_change_0'  # options: price_change_0, logret_0p0015, logret_0p003, logret_0p005
real_label = f'Real-data ({real_data_subdir})'
real_data_root = output_dir / 'real_data'
if real_data_root.exists():
    available_real_runs = sorted([p.name for p in real_data_root.iterdir() if p.is_dir()])
else:
    available_real_runs = []
available_real_runs
real_report_path = real_data_root / real_data_subdir / 'real_data_report.json'
real_meta_path = real_data_root / real_data_subdir / 'real_data_meta.json'


In [ ]:
# Run comparison
from qctsp_hawkes.validation.compare import ComparisonConfig, run_validation_comparison

config = ComparisonConfig(
    T=T,
    mu=mu,
    alpha=alpha,
    beta=beta,
    S=S,
    R=R,
    tau_max=tau_max,
    eta_max=eta_max,
    n_steps=n_steps,
    shots=shots,
    classical_paths=classical_paths,
    seed=seed,
    x0=x0,
    qctsp_backend=qctsp_backend,
    auto_n_steps=auto_n_steps,
    tau_repr_mode=tau_repr_mode,
    tau_edges_mode=tau_edges_mode,
    lambda_ref_mode=lambda_ref_mode,
)

report, report_path = run_validation_comparison(
    config=config,
    output_path=output_dir / 'compare_report.json',
)
report_path


In [ ]:
# Load & summarize
with Path(report_path).open('r', encoding='utf-8') as fp:
    report = json.load(fp)

def _section_summary(section: dict) -> dict:
    return {
        'N_T_mean': section['N_T']['mean'],
        'N_T_var': section['N_T']['var'],
        'I_eta_mean': section['I_eta']['mean'],
        'I_eta_var': section['I_eta']['var'],
        'I_lambda_mean': section['I_lambda']['mean'],
        'I_lambda_var': section['I_lambda']['var'],
        'ks_pvalue_mean': section['ks_pvalue']['mean'],
    }

summary_rows = []
for label in ['classical', 'qctsp']:
    row = {'section': label}
    row.update(_section_summary(report[label]))
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows).set_index('section')
summary_df


### 指標の意味（簡単な解説）
- **N(T)**: 観測時間Tまでに起きたイベント数。多いほどイベントが頻発。
- **I_eta**: 強度状態ηの時間積分（自己励起の総量のような指標）。
- **I_lambda**: 強度λの時間積分（期待される総イベント量に近い）。
- **KS p-value**: 生成過程の整合性チェック。大きいほど整合性が高い傾向。


In [ ]:
# Visualization
classical_nt = np.asarray(report['classical']['N_T']['samples'], dtype=float)
qctsp_nt = np.asarray(report['qctsp']['N_T']['samples'], dtype=float)

bins = np.arange(0, max(classical_nt.max(), qctsp_nt.max()) + 2) - 0.5
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(classical_nt, bins=bins, alpha=0.6, label='classical')
ax.hist(qctsp_nt, bins=bins, alpha=0.6, label='qctsp')
ax.set_xlabel('N(T)')
ax.set_ylabel('count')
ax.set_title('N(T) histogram')
ax.legend()
fig.tight_layout()
nt_path = figure_dir / 'qctsp_vs_classical_nt_hist.png'
fig.savefig(nt_path, dpi=150)
nt_path


In [ ]:
labels = ['classical', 'qctsp']
i_eta_means = [np.mean(report[sec]['I_eta']['samples']) for sec in labels]
i_lambda_means = [np.mean(report[sec]['I_lambda']['samples']) for sec in labels]

x = np.arange(len(labels))
width = 0.35
fig, ax = plt.subplots(figsize=(6, 4))
ax.bar(x - width / 2, i_eta_means, width, label='I_eta')
ax.bar(x + width / 2, i_lambda_means, width, label='I_lambda')
ax.set_xticks(x, labels)
ax.set_ylabel('value')
ax.set_title('Integral comparison')
ax.legend()
fig.tight_layout()
int_path = figure_dir / 'qctsp_vs_classical_integrals.png'
fig.savefig(int_path, dpi=150)
int_path


## Real-Data (OHLCV CSV)
ローカルCSV(`data/btc-usd.csv`)から作成した実データ比較結果を読み込みます。

**イベントの定義**
- OHLCVは「バー」データなので、各バーをイベントとみなす必要があります。
- ここでは `event_mode` を使い、**価格変化が起きたバー**のみをイベントとします。
- しきい値を上げるとイベント数が減り、より「大きな変動のみ」を抽出できます。


**しきい値の考え方（初学者向け）**
- `log_return` のしきい値を上げるほど、**大きな価格変化だけ**をイベントとみなします。
- その結果、イベント数が減り、推定される(μ, α, β)や指標のスケールが変わります。
- 比較の公平性のため、**同じイベント定義**で古典とQCTSPを比較します。


In [ ]:
# Load real-data report
if real_report_path.exists():
    with real_report_path.open('r', encoding='utf-8') as fp:
        real_report = json.load(fp)
    with real_meta_path.open('r', encoding='utf-8') as fp:
        real_meta = json.load(fp)
    real_meta
else:
    print(f'Real-data report not found: {real_report_path}')


In [ ]:
# Summary (real data)
if real_report_path.exists():
    real_summary_rows = []
    for label in ['classical', 'qctsp']:
        row = {'section': label}
        row.update(_section_summary(real_report[label]))
        real_summary_rows.append(row)
    real_summary_df = pd.DataFrame(real_summary_rows).set_index('section')
    real_summary_df


In [ ]:
# Visualization: N(T) histogram (real data)
if real_report_path.exists():
    real_classical_nt = np.asarray(real_report['classical']['N_T']['samples'], dtype=float)
    real_qctsp_nt = np.asarray(real_report['qctsp']['N_T']['samples'], dtype=float)
    bins = np.arange(0, max(real_classical_nt.max(), real_qctsp_nt.max()) + 2) - 0.5
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.hist(real_classical_nt, bins=bins, alpha=0.6, label='classical')
    ax.hist(real_qctsp_nt, bins=bins, alpha=0.6, label='qctsp')
    ax.set_xlabel('N(T)')
    ax.set_ylabel('count')
    ax.set_title(f'{real_label} N(T) histogram')
    ax.legend()
    fig.tight_layout()
    nt_path = figure_dir / f'real_data_{real_data_subdir}_nt_hist.png'
    fig.savefig(nt_path, dpi=150)
    nt_path


In [ ]:
# Visualization: integrals (real data)
if real_report_path.exists():
    labels = ['classical', 'qctsp']
    i_eta_means = [np.mean(real_report[sec]['I_eta']['samples']) for sec in labels]
    i_lambda_means = [np.mean(real_report[sec]['I_lambda']['samples']) for sec in labels]
    x = np.arange(len(labels))
    width = 0.35
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.bar(x - width / 2, i_eta_means, width, label='I_eta')
    ax.bar(x + width / 2, i_lambda_means, width, label='I_lambda')
    ax.set_xticks(x, labels)
    ax.set_ylabel('value')
    ax.set_title(f'{real_label} integral comparison')
    ax.legend()
    fig.tight_layout()
    int_path = figure_dir / f'real_data_{real_data_subdir}_integrals.png'
    fig.savefig(int_path, dpi=150)
    int_path


## Real-Data Sweep Summary（初学者向け）

複数のしきい値で実データ比較を行った場合は、Table 5 を読み込んで一覧できます。
しきい値を上げるほどイベント数が減るため、N(T)などのスケールも小さくなります。


In [ ]:
# Load Table 5 (real-data summary)
table_05_path = output_dir / 'tables' / 'table_05_real_data_summary.csv'
if table_05_path.exists():
    table_05 = pd.read_csv(table_05_path)
    table_05
else:
    print(f'not found: {table_05_path}')


### しきい値スイープの図
以下の図は、しきい値を変えたときのN(T)とI_lambdaの平均値を比較したものです。


In [ ]:
# Show sweep figures if available
sweep_nt = figure_dir / 'real_data_nt_mean_sweep.png'
sweep_il = figure_dir / 'real_data_ilambda_mean_sweep.png'
if sweep_nt.exists():
    display(Image(filename=str(sweep_nt)))
else:
    print(f'not found: {sweep_nt}')
if sweep_il.exists():
    display(Image(filename=str(sweep_il)))
else:
    print(f'not found: {sweep_il}')


## QCTSP Sensitivity（初学者向け）

ここでは **QCTSP側の設定**（`n_steps`, `tau_repr`, `tau_edges`）を変えたときに、
統計指標がどう変化するかを確認します。

- `n_steps` は **何回の保持時間(τ)をサンプルするか**に対応します。
  小さすぎるとイベント数が不足し、N(T)やKSが悪化しやすいです。
- `tau_repr` と `tau_edges` は **τの離散化の仕方**で、
  分布近似の誤差に影響します。


In [ ]:
# Load sensitivity table
table_06_path = output_dir / 'tables' / 'table_06_real_data_sensitivity.csv'
if table_06_path.exists():
    table_06 = pd.read_csv(table_06_path)
    table_06
else:
    print(f'not found: {table_06_path}')


In [ ]:
# Show sensitivity figures
sens_nsteps = figure_dir / 'real_data_nsteps_sensitivity.png'
sens_tau = figure_dir / 'real_data_tau_sensitivity.png'
if sens_nsteps.exists():
    display(Image(filename=str(sens_nsteps)))
else:
    print(f'not found: {sens_nsteps}')
if sens_tau.exists():
    display(Image(filename=str(sens_tau)))
else:
    print(f'not found: {sens_tau}')


## Notes

- `qctsp_backend` を `aer` にすると Qiskit Aer による回路実行を試みます。
- `auto` は環境に応じて Aer か古典サンプラを選択します。
- `tau_repr_mode` は保持時間ビンの代表値の取り方です（midpoint / condexp）。
